In [1]:
import pandas as pd
import numpy as np
import warnings
import sys
import os
% % capture
!pip install autogluon.tabular
!pip install pytest-warnings

warnings.filterwarnings('ignore')


# data preprocess
note  : still need to load in training dataset for data preprocess!

### load in test data

In [2]:
train = pd.read_csv('./tabular-playground-series-aug-2022/train.csv')
test_features = pd.read_csv('./tabular-playground-series-aug-2022/test.csv')
train_features = train.iloc[0:, 0:-1]
all_features = pd.concat([train_features, test_features])

all_features['m_3_missing'] = all_features.measurement_3.isna()
all_features['m_5_missing'] = all_features.measurement_5.isna()
null_features = [
    col for col in all_features.columns if all_features[col].isnull().sum() != 0]
nonull_features = [
    col for col in all_features.columns if col not in null_features]
object_features = [
    col for col in train.columns if train[col].dtypes == 'object']


### fill in missing values

In [3]:
from kuma_utils.preprocessing.imputer import LGBMImputer
!git clone https: // github.com/analokmaus/kuma_utils.git
sys.path.append("kuma_utils/")


fatal: destination path 'kuma_utils' already exists and is not an empty directory.


In [4]:
display(all_features['product_code'].unique())
df_A = all_features[all_features['product_code'] == 'A']
df_B = all_features[all_features['product_code'] == 'B']
df_C = all_features[all_features['product_code'] == 'C']
df_D = all_features[all_features['product_code'] == 'D']
df_E = all_features[all_features['product_code'] == 'E']
df_F = all_features[all_features['product_code'] == 'F']
df_G = all_features[all_features['product_code'] == 'G']
df_H = all_features[all_features['product_code'] == 'H']
df_I = all_features[all_features['product_code'] == 'I']


array(['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I'], dtype=object)

In [5]:
lgbm_imtr = LGBMImputer(cat_features=object_features, n_iter=50)

# train dataset
train_iterimp_A = lgbm_imtr.fit_transform(df_A[null_features])
train_iterimp_B = lgbm_imtr.fit_transform(df_B[null_features])
train_iterimp_C = lgbm_imtr.fit_transform(df_C[null_features])
train_iterimp_D = lgbm_imtr.fit_transform(df_D[null_features])
train_iterimp_E = lgbm_imtr.fit_transform(df_E[null_features])

# test dataset
test_iterimp_F = lgbm_imtr.fit_transform(df_F[null_features])
test_iterimp_G = lgbm_imtr.fit_transform(df_G[null_features])
test_iterimp_H = lgbm_imtr.fit_transform(df_H[null_features])
test_iterimp_I = lgbm_imtr.fit_transform(df_I[null_features])


  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

In [6]:
all_features_ = pd.concat([train_iterimp_A, train_iterimp_B, train_iterimp_C, train_iterimp_D,
                           train_iterimp_E, test_iterimp_F, test_iterimp_G, test_iterimp_H, test_iterimp_I
                           ])
all_features = pd.concat(
    [all_features_, all_features[nonull_features]], axis=1)
print("Missing values in train dataset after pre-peocessing is: ",
      format(all_features.isna().sum().sum()))


Missing values in train dataset after pre-peocessing is:  0


### choose columns

In [7]:
all_features['a_2*3'] = all_features['attribute_2'] * \
    all_features['attribute_3']

meas_gr1_cols = [f"measurement_{i:d}" for i in list(
    range(3, 5)) + list(range(9, 17))]
all_features['meas_gr1_avg'] = np.mean(all_features[meas_gr1_cols], axis=1)
all_features['meas_gr1_std'] = np.std(all_features[meas_gr1_cols], axis=1)

meas_gr2_cols = [f"measurement_{i:d}" for i in list(range(5, 9))]
all_features['meas_gr2_avg'] = np.mean(all_features[meas_gr2_cols], axis=1)
all_features['meas_gr2_avg'] = np.mean(all_features[meas_gr2_cols], axis=1)

all_features['meas17/meas_gr2_avg'] = all_features['measurement_17'] / \
    all_features['meas_gr2_avg']


In [8]:
cols_to_use = ['attribute_0', 'measurement_0', 'measurement_1', 'measurement_2', 'attribute_1', 'm_3_missing', 'm_5_missing',
               'meas_gr1_avg', 'meas_gr1_std', 'a_2*3', 'loading', 'measurement_17', 'meas17/meas_gr2_avg']
all_features = all_features[cols_to_use]
all_features = pd.get_dummies(
    all_features, columns=['attribute_0', 'attribute_1'])


# load in model and test

In [9]:
from autogluon.tabular import TabularPredictor, TabularDataset


In [10]:
test = all_features.iloc[26570:]  # 26570 是 labels.shape[0]
test = TabularDataset(test)


In [11]:
predictor = TabularPredictor.load("AutogluonModels/ag-20230107_133316/")
pred_test = predictor.predict_proba(test, as_pandas=False, as_multiclass=False)
list = [i for i in range(26570, len(pred_test) + 26570)
        ]  # 26570 是 train.shape[0]
sub = pd.DataFrame({'id': list, 'failure': pred_test})
sub.to_csv('submission.csv', index=False)
display(sub)


,id,failure
0,26570,0.197063
1,26571,0.168523
2,26572,0.194533
3,26573,0.187086
4,26574,0.309533
...,...,...
20770,47340,0.236613
20771,47341,0.136116
20772,47342,0.148281
20773,47343,0.223224
